In [481]:
import yaml
import sagemaker
import boto3
import pandas as pd
import os
import json
from sklearn.model_selection import train_test_split
from time import gmtime, strftime
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.sklearn.model import SKLearnModel
from sagemaker.predictor import Predictor

In [41]:
SETTING_FILE_PATH = "../config/settings.yaml"
DATA_FOLDER_PATH = "../avazu-ctr-prediction"

# AWS リソース設定
with open(SETTING_FILE_PATH) as file:
    aws_info = yaml.safe_load(file)
        
sess = sagemaker.Session()

role = aws_info['aws']['sagemaker']['role']
bucket = aws_info['aws']['sagemaker']['s3bucket']
region = aws_info['aws']['sagemaker']['region']

## モデル学習


In [42]:
# train, validation データを用意
df_train = pd.read_csv(os.path.join(DATA_FOLDER_PATH, "train_partial"), dtype="object")


In [43]:

# S3にアップロード
prefix = 'model-monitoring'

train_file = "train.csv"

df_train.to_csv(train_file, index=False)

s3_resource_bucket = boto3.Session().resource("s3").Bucket(bucket)
s3_resource_bucket.Object(os.path.join(prefix, "train", train_file)).upload_file(train_file)


In [44]:
output_location = f"s3://{bucket}/{prefix}/output"

s3_train_data = f"s3://{bucket}/{prefix}/train/{train_file}"

In [329]:
timestamp = strftime("%Y%m%d-%H-%M-%S", gmtime())
job_name = "model-training-job" + timestamp

hyperparameters = {"alpha": 0.00001, "eta0": 2.0}
enable_local_mode_training = False

if enable_local_mode_training:
    train_instance_type = "local"
    inputs = {"train": f"file://{train_file}"}
else:
    train_instance_type = "ml.m5.large"
    inputs = {"train": s3_train_data}

estimator_parameters = {
    "entry_point": "trainer.py",
    "source_dir": "model",
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": train_instance_type,
    "instance_count": 1,
    "hyperparameters": hyperparameters,
    "output_path": output_location,
    "role": role,
    "base_job_name": job_name,
}

model_estimator = SKLearn(**estimator_parameters)
model_estimator.fit(inputs)


2022-11-24 10:38:19 Starting - Starting the training job...
2022-11-24 10:38:41 Starting - Preparing the instances for trainingProfilerReport-1669286298: InProgress
.........
2022-11-24 10:40:20 Downloading - Downloading input data...
2022-11-24 10:40:46 Training - Training image download completed. Training in progress..2022-11-24 10:40:50,944 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2022-11-24 10:40:50,947 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-11-24 10:40:50,956 sagemaker_sklearn_container.training INFO     Invoking user training script.
2022-11-24 10:40:51,191 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-11-24 10:40:51,203 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-11-24 10:40:51,215 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2022-11-24 10:40:51,224 sagemaker-trai

## SageMaker Model Monitor

In [349]:
    
timestamp = strftime("%Y%m%d-%H-%M-%S", gmtime())
    
model = SKLearnModel(
    role=role,
    model_data=model_estimator.model_data,
    framework_version="0.23-1",
    py_version="py3",
    source_dir="model",
    entry_point="inference.py",
    sagemaker_session=sess
)

model_name = "{}-{}".format("monitoring-model", timestamp)


sess.create_model(
    model_name,
    role,
    model.prepare_container_def(
        instance_type='ml.t2.medium'
    )
)

'model-20221124-10-57-38'

In [419]:
from sagemaker.model_monitor import DataCaptureConfig

endpoint_name = 'model-monitor-endopoint' + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
print("EndpointName={}".format(endpoint_name))

data_capture_prefix = "{}/datacapture".format(prefix)
s3_capture_upload_path = "s3://{}/{}".format(bucket, data_capture_prefix)

# データキャプチャーの設定
data_capture_config = DataCaptureConfig(
                        enable_capture = True,
                        sampling_percentage=50,
                        destination_s3_uri=s3_capture_upload_path,
                        kms_key_id=None,
                        capture_options=["REQUEST"],
                        csv_content_types=["text/csv"],
                        json_content_types=["application/json"]
                        )

predictor = model.deploy(initial_instance_count=1,
                instance_type="ml.t2.medium",
                endpoint_name=endpoint_name,
                data_capture_config=data_capture_config
                )

predictor = Predictor(endpoint_name=endpoint_name)

EndpointName=model-monitor-endopoint2022-11-25-01-20-23
------------!

In [443]:
runtime = boto3.Session().client('sagemaker-runtime')

with open('test_partial.csv') as f:
    for i, line in enumerate(f):
        response = runtime.invoke_endpoint(EndpointName=endpoint_name, 
                                   ContentType='text/csv', 
                                   Body=line,
                                  Accept='application/json')
        if i == 5000:
            break

In [422]:
response_dict = json.loads(response['Body'].read().decode("utf-8"))
print(json.dumps(response_dict, indent=4))

[
    [
        0.9984024292532515,
        0.0015975707467485085
    ]
]


In [436]:
# fileチェック
s3_client = boto3.Session().client('s3')
current_endpoint_capture_prefix = '{}/{}'.format(data_capture_prefix, endpoint_name)
result = s3_client.list_objects(Bucket=bucket, Prefix=current_endpoint_capture_prefix)
capture_files = [capture_file.get("Key") for capture_file in result.get('Contents')]
print("Found Capture Files:")
print("\n ".join(capture_files))


Found Capture Files:
model-monitoring/datacapture/model-monitor-endopoint2022-11-25-01-20-23/AllTraffic/2022/11/25/01/26-44-116-4fc56cc8-d743-438f-9220-9b391ae43676.jsonl
 model-monitoring/datacapture/model-monitor-endopoint2022-11-25-01-20-23/AllTraffic/2022/11/25/01/37-20-753-04d1982c-0320-42a4-9b56-d099478b1e53.jsonl
 model-monitoring/datacapture/model-monitor-endopoint2022-11-25-01-20-23/AllTraffic/2022/11/25/01/38-20-865-2dcb9a51-66c8-48b6-9550-bcf2d7cfb664.jsonl
 model-monitoring/datacapture/model-monitor-endopoint2022-11-25-01-20-23/AllTraffic/2022/11/25/02/09-15-551-dc50f2f7-826e-4dd9-adc5-3b8eb56be771.jsonl
 model-monitoring/datacapture/model-monitor-endopoint2022-11-25-01-20-23/AllTraffic/2022/11/25/02/10-15-599-d92fbdeb-8af9-41d3-b4f4-2c0be4284bc1.jsonl
 model-monitoring/datacapture/model-monitor-endopoint2022-11-25-01-20-23/AllTraffic/2022/11/25/02/11-15-799-d54f039c-e8d9-4cd6-b8d8-6ddc8b1d5289.jsonl
 model-monitoring/datacapture/model-monitor-endopoint2022-11-25-01-20-23/A

In [437]:
def get_obj_body(obj_key):
    return s3_client.get_object(Bucket=bucket, Key=obj_key).get("Body").read().decode("utf-8")


capture_file = get_obj_body(capture_files[-1])
print(capture_file[:2000])

{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"2.9187456379418363e+18,0,14102119,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,da0e56ab,5096d134,1,0,17262,320,50,1872,3,39,-1,23\n","encoding":"CSV"}},"eventMetadata":{"eventId":"675bd69a-ac46-4297-b550-e7e0a4a80ca1","inferenceTime":"2022-11-25T02:13:16Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"8.008605692250388e+18,0,14102814,1005,0,85f751fd,c4e18dd6,50e219e0,73206397,2347f47a,8ded1f7a,98cd0435,6c6bc2de,ecb851b2,1,0,23143,320,50,2665,0,35,100148,221\n","encoding":"CSV"}},"eventMetadata":{"eventId":"d5c4cebd-a7de-4939-a763-87ed68d0c698","inferenceTime":"2022-11-25T02:13:16Z"},"eventVersion":"0"}
{"captureData":{"endpointInput":{"observedContentType":"text/csv","mode":"INPUT","data":"6.623343463520788e+18,0,14102808,1005,0,85f751fd,c4e18dd6,50e219e0,e2fcccd2,5c5a694b,0f2161f8,a99f214a,fe84bcaf,be74e6fe,1,

In [362]:
import json

print(json.dumps(json.loads(capture_file.split("\n")[0]), indent=2))

{
  "captureData": {
    "endpointInput": {
      "observedContentType": "text/csv",
      "mode": "INPUT",
      "data": "1.0807444605793397e+19,0,14102405,1005,1,856e6d3f,58a89a43,f028772b,ecad2386,7801e8d9,07d7df22,a99f214a,66a9be9c,684581ce,1,0,21981,320,50,2532,0,679,100077,48\n",
      "encoding": "CSV"
    }
  },
  "eventMetadata": {
    "eventId": "4a8a5ed9-6393-432c-8295-35b272ade59f",
    "inferenceTime": "2022-11-24T11:07:53Z"
  },
  "eventVersion": "0"
}


In [428]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type="ml.m5.large",
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)
baseline_results_uri = f"s3://{bucket}/{prefix}/model_monitor/results"
my_default_monitor.suggest_baseline(
    baseline_dataset=s3_train_data,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)


Job Name:  baseline-suggestion-job-2022-11-25-01-28-38-620
Inputs:  [{'InputName': 'baseline_dataset_input', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ctr-prediction/model-monitoring/train/train.csv', 'LocalPath': '/opt/ml/processing/input/baseline_dataset_input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'monitoring_output', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://ctr-prediction/model-monitoring/model_monitor/results', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
................................2022-11-25 01:33:47,129 - matplotlib.font_manager - INFO - Generating new fontManager, this may take some time...
2022-11-25 01:33:48.739437: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
202

In [439]:

baseline_job = my_default_monitor.latest_baselining_job
schema_df = pd.json_normalize(baseline_job.baseline_statistics().body_dict["features"])

schema_df.head(10)


,name,inferred_type,string_statistics.common.num_present,string_statistics.common.num_missing,string_statistics.distinct_count,numerical_statistics.common.num_present,numerical_statistics.common.num_missing,numerical_statistics.mean,numerical_statistics.sum,numerical_statistics.std_dev,numerical_statistics.min,numerical_statistics.max,numerical_statistics.distribution.kll.buckets,numerical_statistics.distribution.kll.sketch.parameters.c,numerical_statistics.distribution.kll.sketch.parameters.k,numerical_statistics.distribution.kll.sketch.data,string_statistics.distribution.categorical.buckets
0,id,String,323432.0,0.0,329471.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,click,Integral,NaN,NaN,NaN,323432.0,0.0,1.686784e-01,5.455600e+04,0.374468,0.0,1.0,"[{'lower_bound': 0.0, 'upper_bound': 0.1, 'cou...",0.64,2048.0,"[[], [], [], [1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",NaN
2,hour,Integral,NaN,NaN,NaN,323432.0,0.0,1.410256e+07,4.561219e+12,296.652135,14102100.0,14103023.0,"[{'lower_bound': 14102100.0, 'upper_bound': 14...",0.64,2048.0,"[[], [], [], [14103023.0, 14102100.0, 14102100...",NaN
3,C1,Integral,NaN,NaN,NaN,323432.0,0.0,1.004969e+03,3.250391e+08,1.098521,1001.0,1012.0,"[{'lower_bound': 1001.0, 'upper_bound': 1002.1...",0.64,2048.0,"[[], [], [], [1012.0, 1002.0, 1002.0, 1002.0, ...",NaN
4,banner_pos,Integral,NaN,NaN,NaN,323432.0,0.0,2.886356e-01,9.335400e+04,0.506276,0.0,7.0,"[{'lower_bound': 0.0, 'upper_bound': 0.7, 'cou...",0.64,2048.0,"[[], [], [], [2.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0....",NaN
5,site_id,String,323432.0,0.0,1994.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,site_domain,String,323432.0,0.0,2088.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,site_category,String,323432.0,0.0,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"[{'value': 'c0dd3be3', 'count': 331}, {'value'..."
8,app_id,String,323432.0,0.0,2080.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,app_domain,String,323432.0,0.0,124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [368]:
kll_buckets = schema_df[schema_df.name == 'hour']['numerical_statistics.distribution.kll.buckets'].tolist()[0]
for kll_bucket in kll_buckets:
    print(kll_bucket)

{'lower_bound': 14102100.0, 'upper_bound': 14102192.3, 'count': 33232.0}
{'lower_bound': 14102192.3, 'upper_bound': 14102284.6, 'count': 42312.0}
{'lower_bound': 14102284.6, 'upper_bound': 14102376.9, 'count': 31112.0}
{'lower_bound': 14102376.9, 'upper_bound': 14102469.2, 'count': 26464.0}
{'lower_bound': 14102469.2, 'upper_bound': 14102561.5, 'count': 27056.0}
{'lower_bound': 14102561.5, 'upper_bound': 14102653.8, 'count': 30800.0}
{'lower_bound': 14102653.8, 'upper_bound': 14102746.1, 'count': 25736.0}
{'lower_bound': 14102746.1, 'upper_bound': 14102838.4, 'count': 42344.0}
{'lower_bound': 14102838.4, 'upper_bound': 14102930.7, 'count': 30680.0}
{'lower_bound': 14102930.7, 'upper_bound': 14103023.0, 'count': 33696.0}


In [440]:
constraints_df = pd.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

,name,inferred_type,completeness,num_constraints.is_non_negative,string_constraints.domains
0,id,String,1.0,NaN,NaN
1,click,Integral,1.0,True,NaN
2,hour,Integral,1.0,True,NaN
3,C1,Integral,1.0,True,NaN
4,banner_pos,Integral,1.0,True,NaN
5,site_id,String,1.0,NaN,NaN
6,site_domain,String,1.0,NaN,NaN
7,site_category,String,1.0,NaN,"[c0dd3be3, 28905ebd, 70fb0e29, a818d37a, 76b29..."
8,app_id,String,1.0,NaN,NaN
9,app_domain,String,1.0,NaN,NaN


In [431]:
from sagemaker.model_monitor import CronExpressionGenerator

monitor_schedule_name = 'ctr-prediction-monitoring3'
s3_report_path = f's3://{bucket}/model_monitor/monitoring_report'
my_default_monitor.create_monitoring_schedule(
    monitor_schedule_name=monitor_schedule_name,
    endpoint_input=predictor.endpoint,
    output_s3_uri=s3_report_path,
    statistics=my_default_monitor.baseline_statistics(),
    constraints=my_default_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
    enable_cloudwatch_metrics=True,
)

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [441]:
desc_schedule_result = my_default_monitor.describe_schedule()
print("Schedule status: {}".format(desc_schedule_result["MonitoringScheduleStatus"]))


Schedule status: Scheduled


In [466]:
client = boto3.client('sagemaker', region_name=region)


mon_executions = client.list_monitoring_executions(MonitoringScheduleName=monitor_schedule_name, MaxResults=1)


for execution_summary in mon_executions['MonitoringExecutionSummaries']:
    job_name = execution_summary['ProcessingJobArn'].split('/')[1]
    
    print("ProcessingJob: {}".format(execution_summary['ProcessingJobArn'].split('/')[1]))
    
    print('MonitoringExecutionStatus: {} \n'.format(execution_summary['MonitoringExecutionStatus']))

    desc_analytics_job_result=client.describe_processing_job(ProcessingJobName=job_name)
    report_uri=desc_analytics_job_result['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']
print('Report Uri: {}'.format(report_uri))

ProcessingJob: model-monitoring-202211251500-b57b624f69b7ff79457dd3b2
MonitoringExecutionStatus: CompletedWithViolations 

Report Uri: s3://ctr-prediction/model_monitor/monitoring_report/model-monitor-endopoint2022-11-25-01-20-23/ctr-prediction-monitoring3/2022/11/25/15


In [289]:
!wget https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/sagemaker_model_monitor/visualization/utils.py
!pip install Jinja2

import os
from IPython.display import HTML, display
from sagemaker.s3 import S3Downloader
from sagemaker.model_monitor import MonitoringExecution
import utils as mu

--2022-11-24 13:19:42--  https://raw.githubusercontent.com/awslabs/amazon-sagemaker-examples/master/sagemaker_model_monitor/visualization/utils.py
raw.githubusercontent.com (raw.githubusercontent.com) をDNSに問いあわせています... 2606:50c0:8003::154, 2606:50c0:8002::154, 2606:50c0:8001::154, ...
raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443 に接続しています... 接続しました。
HTTP による接続要求を送信しました、応答を待っています... 200 OK
長さ: 13024 (13K) [text/plain]
`utils.py.1' に保存中

utils.py.1          100%[===================>]  12.72K  --.-KB/s 時間 0s         

2022-11-24 13:19:42 (29.9 MB/s) - `utils.py.1' へ保存完了 [13024/13024]



In [475]:
baseline_statistics = json.loads(S3Downloader.read_file('s3://ctr-prediction/model_monitor/monitoring_report/model-monitor-endopoint2022-11-25-01-20-23/ctr-prediction-monitoring3/2022/11/25/03/statistics.json')) 
execution_statistics = json.loads(S3Downloader.read_file('s3://ctr-prediction/model_monitor/monitoring_report/model-monitor-endopoint2022-11-25-01-20-23/ctr-prediction-monitoring3/2022/11/25/03/statistics.json'))
violations = json.loads(S3Downloader.read_file('s3://ctr-prediction/model_monitor/monitoring_report/model-monitor-endopoint2022-11-25-01-20-23/ctr-prediction-monitoring3/2022/11/25/03/constraint_violations.json'))



In [ ]:
MonitoringExecution.from_processing_arn(sagemaker_session=sess, processing_job_arn=processing_job_arn)


In [471]:
mon_executions = client.list_monitoring_executions(MonitoringScheduleName=monitor_schedule_name, MaxResults=1)

for execution_summary in mon_executions['MonitoringExecutionSummaries']:
    # processing_job_arn = execution_summary['ProcessingJobArn']
    processing_job_arn = 'arn:aws:sagemaker:ap-northeast-1:547760918250:processing-job/model-monitoring-202211250300-3ec939a8312cbe15ebf43e3c'
    execution = MonitoringExecution.from_processing_arn(sagemaker_session=sess, processing_job_arn=processing_job_arn)
    exec_inputs = {inp['InputName']: inp for inp in execution.describe()['ProcessingInputs']}
    exec_results = execution.output.destination

    baseline_statistics_filepath = exec_inputs['baseline']['S3Input']['S3Uri'] if 'baseline' in exec_inputs else None
    execution_statistics_filepath = os.path.join(exec_results, 'statistics.json')
    violations_filepath = os.path.join(exec_results, 'constraint_violations.json')

    baseline_statistics = json.loads(S3Downloader.read_file(baseline_statistics_filepath)) if baseline_statistics_filepath is not None else None
    print(execution_statistics_filepath)
    print(baseline_statistics_filepath)
    print(violations_filepath )
    execution_statistics = json.loads(S3Downloader.read_file(execution_statistics_filepath))
    violations = json.loads(S3Downloader.read_file(violations_filepath))['violations']

mu.show_violation_df(baseline_statistics=baseline_statistics, latest_statistics=execution_statistics, violations=violations)

s3://ctr-prediction/model_monitor/monitoring_report/model-monitor-endopoint2022-11-25-01-20-23/ctr-prediction-monitoring3/2022/11/25/03/statistics.json
s3://ctr-prediction/model-monitoring/model_monitor/results/statistics.json
s3://ctr-prediction/model_monitor/monitoring_report/model-monitor-endopoint2022-11-25-01-20-23/ctr-prediction-monitoring3/2022/11/25/03/constraint_violations.json


,data_type,completeness,baseline_drift,categorical_values
C1,Integral,100.00%,N/A,N/A
C14,Integral,100.00%,N/A,N/A
C15,Integral,100.00%,N/A,N/A
C16,Integral,100.00%,N/A,N/A
C17,Integral,100.00%,N/A,N/A
C18,Integral,100.00%,N/A,N/A
C19,Integral,100.00%,N/A,N/A
C20,Integral,100.00%,N/A,N/A
C21,Integral,100.00%,N/A,N/A
app_category,String,100.00%,N/A,N/A


## SageMaker Clarify

In [376]:
from sagemaker import clarify

clarify_processor = clarify.SageMakerClarifyProcessor(
    role=role, instance_count=1, instance_type="ml.m5.xlarge", sagemaker_session=sess
)

In [377]:
bias_report_output_path = "s3://{}/{}/clarify-bias".format(bucket, prefix)
bias_data_config = clarify.DataConfig(
    s3_data_input_path=s3_train_data,
    s3_output_path=bias_report_output_path,
    label="click",
    headers=df_train.columns.to_list(),
    dataset_type="text/csv",
)

In [378]:
model_config = clarify.ModelConfig(
    model_name=model_name,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    accept_type="text/csv",
    content_type="text/csv"
)

In [379]:
predictions_config = clarify.ModelPredictedLabelConfig(probability_threshold=0.5)

In [382]:
bias_config = clarify.BiasConfig(
    label_values_or_threshold=[1], facet_name="banner_pos", facet_values_or_threshold=[0], group_name="C1"
)

In [383]:
clarify_processor.run_bias(
    data_config=bias_data_config,
    bias_config=bias_config,
    model_config=model_config,
    model_predicted_label_config=predictions_config,
    pre_training_methods="all",
    post_training_methods="all",
)


Job Name:  Clarify-Bias-2022-11-24-11-54-13-841
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ctr-prediction/model-monitoring/train/train.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ctr-prediction/model-monitoring/clarify-bias/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://ctr-prediction/model-monitoring/clarify-bias', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
..............................2022-11-24 11:59:09,266 logging.conf not found when configuring logging, using def

In [414]:
shap_config = clarify.SHAPConfig(baseline=None,
                                 num_samples=35,
                                 agg_method='mean_abs',
                                 save_local_shap_values=False)

explainability_output_path = 's3://{}/{}/clarify-explainability'.format(bucket, prefix)
explainability_data_config = clarify.DataConfig(s3_data_input_path=s3_train_data,
                                s3_output_path=explainability_output_path,
                                label='click',
                                headers=df_train.columns.to_list(),
                                dataset_type='text/csv')

In [416]:
clarify_processor.run_explainability(data_config=explainability_data_config,
                                     model_config=model_config,
                                     explainability_config=shap_config)


Job Name:  Clarify-Explainability-2022-11-24-14-00-23-377
Inputs:  [{'InputName': 'dataset', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ctr-prediction/model-monitoring/train/train.csv', 'LocalPath': '/opt/ml/processing/input/data', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'analysis_config', 'AppManaged': False, 'S3Input': {'S3Uri': 's3://ctr-prediction/model-monitoring/clarify-explainability/analysis_config.json', 'LocalPath': '/opt/ml/processing/input/config', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'analysis_result', 'AppManaged': False, 'S3Output': {'S3Uri': 's3://ctr-prediction/model-monitoring/clarify-explainability', 'LocalPath': '/opt/ml/processing/output', 'S3UploadMode': 'EndOfJob'}}]
...............................2022-11-24 14:05:21,618 logging.conf not found when